In [1]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Try to match country names from DOL data (Sweat & Toil) to World Bank data (indicators)
Use DOL countries table as the master list

In [2]:
net_migration = pd.read_csv('data/API_SM.POP.NETM_DS2_en_csv_v2.csv', header=2)
countries = pd.read_csv('data/Countries.csv', index_col=0)

C and S appear to be typos in DOL data and are removed.

In [3]:
countries = countries[(countries.name !='C') & (countries.name !='S')]

In [4]:
net_migration.head() # most recent data is 2012
countries.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,Unnamed: 61
0,Aruba,ABW,Net migration,SM.POP.NETM,NaN,NaN,-4323.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1253.0,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,Net migration,SM.POP.NETM,NaN,NaN,-20000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,448007.0,NaN,NaN,NaN,NaN,NaN
2,Angola,AGO,Net migration,SM.POP.NETM,NaN,NaN,-135000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,87322.0,NaN,NaN,NaN,NaN,NaN
3,Albania,ALB,Net migration,SM.POP.NETM,NaN,NaN,-99.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-93425.0,NaN,NaN,NaN,NaN,NaN
4,Andorra,AND,Net migration,SM.POP.NETM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,id,name,num_territories,region,region_id
0,1,Afghanistan,0,Asia & the Pacific,1
1,10,Bangladesh,0,Asia & the Pacific,1
2,13,Bhutan,0,Asia & the Pacific,1
3,21,Burma,0,Asia & the Pacific,1
4,24,Cambodia,0,Asia & the Pacific,1


Join datasets to see which countries in DOL data do not match with World Bank data

In [5]:
joined =countries.merge(net_migration[['Country Name', 'Indicator Name', '2012']], how='left', left_on='name', right_on='Country Name')
joined.shape

(146, 8)

In [6]:
mismatch = joined[pd.isnull(joined['2012'])]
mismatch.shape
mismatch

(34, 8)

,id,name,num_territories,region,region_id,Country Name,Indicator Name,2012
3,21,Burma,0,Asia & the Pacific,1,NaN,NaN,NaN
6,30,Christmas Island,0,Asia & the Pacific,1,NaN,NaN,NaN
7,31,Cocos (Keeling) Islands,0,Asia & the Pacific,1,NaN,NaN,NaN
8,36,Cook Islands,0,Asia & the Pacific,1,NaN,NaN,NaN
10,59,Heard and McDonald Islands,0,Asia & the Pacific,1,NaN,NaN,NaN
20,95,Niue,0,Asia & the Pacific,1,NaN,NaN,NaN
21,96,Norfolk Island,0,Asia & the Pacific,1,NaN,NaN,NaN
22,97,North Korea,0,Asia & the Pacific,1,NaN,NaN,NaN
32,130,Tokelau,0,Asia & the Pacific,1,NaN,NaN,NaN
35,135,Tuvalu,0,Asia & the Pacific,1,Tuvalu,Net migration,NaN


Create name_normalized column in countries table, containing names that match World Bank data.   

Manually matching those that were unmatched. Some countries are not represented in World Bank data or have missing values in specified dataset.

In [7]:
countries.shape
countries['name_normalized'] = countries['name'] 
countries.loc[countries.name == 'Burma','name_normalized'] = 'Myanmar'
countries.loc[countries.name == 'North Korea','name_normalized'] = "Korea, Dem. People’s Rep."
countries.loc[countries.name == 'Macedonia','name_normalized'] = 'Macedonia, FYR'
countries.loc[countries.name == 'Moldova, Republic of','name_normalized'] = 'Moldova'
countries.loc[countries.name == 'Russia','name_normalized'] = 'Russian Federation'
countries.loc[countries.name == 'Egypt','name_normalized'] = 'Egypt, Arab Rep.'
countries.loc[countries.name == 'Iran','name_normalized'] = 'Iran, Islamic Rep.'
countries.loc[countries.name == 'West Bank and the Gaza Strip','name_normalized'] = 'West Bank and Gaza'
countries.loc[countries.name == 'Yemen','name_normalized'] = 'Yemen, Rep.'
countries.loc[countries.name == 'Congo, Democratic Republic of the','name_normalized'] = 'Congo, Dem. Rep.'
countries.loc[countries.name == 'Congo, Republic of the','name_normalized'] = 'Congo, Rep.'
countries.loc[countries.name == 'Gambia','name_normalized'] = 'Gambia, The'
countries.loc[countries.name == 'Saint Lucia','name_normalized'] = 'St. Lucia'
countries.loc[countries.name == 'Saint Vincent and the Grenadines','name_normalized'] = 'St. Vincent and the Grenadines'
countries.loc[countries.name == 'Venezuela','name_normalized'] = 'Venezuela, RB'

countries.shape

(146, 5)

(146, 6)

Join again to see if we get expected results.

In [8]:
joined2 = countries.merge(net_migration[['Country Name', 'Indicator Name', '2012']], how='inner', left_on='name_normalized', right_on='Country Name')
joined2.shape
mismatch2 = joined2[pd.isnull(joined2['2012'])]
mismatch2.shape
mismatch2

(131, 9)

(4, 9)

,id,name,num_territories,region,region_id,name_normalized,Country Name,Indicator Name,2012
28,135,Tuvalu,0,Asia & the Pacific,1,Tuvalu,Tuvalu,Net migration,NaN
36,19,British Virgin Islands,0,Europe & Eurasia,2,British Virgin Islands,British Virgin Islands,Net migration,NaN
38,70,Kosovo,0,Europe & Eurasia,2,Kosovo,Kosovo,Net migration,NaN
111,40,Dominica,0,Latin America & the Caribbean,5,Dominica,Dominica,Net migration,NaN


In [9]:
countries.to_csv('data/Countries_normalized.csv')

Combine normalized_countries table with all World Bank indicators

In [10]:
net_exports = pd.read_csv('data/API_NE.EXP.GNFS.ZS_DS2_en_csv_v2.csv', header=2)
net_exports.head(10) # most recent data is 2016

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,Unnamed: 61
0,Aruba,ABW,Exports of goods and services (% of GDP),NE.EXP.GNFS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,66.260673,63.786331,61.502448,70.772061,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,Exports of goods and services (% of GDP),NE.EXP.GNFS.ZS,4.132233,4.453443,4.878051,9.171601,8.888893,11.258279,...,17.992975,14.707589,10.023850,6.107274,5.516850,6.307895,6.567700,7.002117,6.896250,NaN
2,Angola,AGO,Exports of goods and services (% of GDP),NE.EXP.GNFS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,76.318427,54.907688,62.387739,65.352709,62.313398,55.688943,47.998691,33.927064,33.019892,NaN
3,Albania,ALB,Exports of goods and services (% of GDP),NE.EXP.GNFS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,29.590228,29.600867,32.440017,34.013685,33.351643,35.449059,28.230908,27.259171,28.739835,NaN
4,Andorra,AND,Exports of goods and services (% of GDP),NE.EXP.GNFS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Arab World,ARB,Exports of goods and services (% of GDP),NE.EXP.GNFS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,56.971029,47.429042,49.967211,54.841197,55.741359,53.924627,51.254715,44.100301,40.964208,NaN
6,United Arab Emirates,ARE,Exports of goods and services (% of GDP),NE.EXP.GNFS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,78.867889,79.653318,77.713035,89.719801,100.261456,100.549138,99.100149,100.405575,103.821198,NaN
7,Argentina,ARG,Exports of goods and services (% of GDP),NE.EXP.GNFS.ZS,7.604049,5.994947,4.691843,7.890454,5.563716,6.225874,...,22.060900,19.560985,18.933823,18.449209,16.237859,14.617173,14.409819,11.008468,12.711001,NaN
8,Armenia,ARM,Exports of goods and services (% of GDP),NE.EXP.GNFS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,15.047103,15.474469,20.830095,23.756853,27.566722,28.359126,28.565139,29.796196,33.142750,NaN
9,American Samoa,ASM,Exports of goods and services (% of GDP),NE.EXP.GNFS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,113.854352,78.171091,63.194444,58.362369,74.844720,71.361502,66.300940,66.302652,NaN,NaN


In [11]:
net_imports = pd.read_csv('data/API_NE.IMP.GNFS.ZS_DS2_en_csv_v2.csv', header=2)
net_imports.head(10) # most recent data is 2016

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,Unnamed: 61
0,Aruba,ABW,Imports of goods and services (% of GDP),NE.IMP.GNFS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,74.926615,76.276578,74.816343,86.436355,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,Imports of goods and services (% of GDP),NE.IMP.GNFS.ZS,7.024793,8.097166,9.349593,16.863910,18.055555,21.412803,...,55.072546,42.199228,44.943478,44.193948,39.142397,49.757775,45.773877,48.801615,49.024977,NaN
2,Angola,AGO,Imports of goods and services (% of GDP),NE.IMP.GNFS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,51.226551,55.408684,42.949968,42.162962,39.739433,39.380009,42.211533,36.142881,32.676961,NaN
3,Albania,ALB,Imports of goods and services (% of GDP),NE.IMP.GNFS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,56.437144,53.758696,53.024544,56.749178,51.986868,53.480552,47.224798,44.520190,45.580111,NaN
4,Andorra,AND,Imports of goods and services (% of GDP),NE.IMP.GNFS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Arab World,ARB,Imports of goods and services (% of GDP),NE.IMP.GNFS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,40.878510,42.296407,40.650017,39.863928,41.051073,41.858024,43.275111,46.894119,44.791048,NaN
6,United Arab Emirates,ARE,Imports of goods and services (% of GDP),NE.IMP.GNFS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,69.645655,73.808626,79.032891,81.631099,85.313460,85.228854,89.022846,96.023909,101.439798,NaN
7,Argentina,ARG,Imports of goods and services (% of GDP),NE.IMP.GNFS.ZS,7.604047,5.994945,9.383683,7.890452,5.563714,4.150583,...,18.341773,14.496142,16.037190,16.756946,14.288683,14.716756,14.001315,11.848140,13.403047,NaN
8,Armenia,ARM,Imports of goods and services (% of GDP),NE.IMP.GNFS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,40.655759,43.004401,45.317192,47.354012,48.400022,48.197069,47.046546,41.961313,43.306254,NaN
9,American Samoa,ASM,Imports of goods and services (% of GDP),NE.IMP.GNFS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,133.925400,82.005900,93.229167,97.386760,99.844720,100.312989,106.269592,103.900156,NaN,NaN


In [12]:
youth_literacy = pd.read_csv('data/API_SE.ADT.1524.LT.ZS_DS2_en_csv_v2.csv', header=2)
youth_literacy.head(20) # too many missing values

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,Unnamed: 61
0,Aruba,ABW,"Literacy rate, youth total (% of people ages 1...",SE.ADT.1524.LT.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,99.13841,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,"Literacy rate, youth total (% of people ages 1...",SE.ADT.1524.LT.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,46.99005,NaN,NaN,NaN,NaN,NaN,NaN
2,Angola,AGO,"Literacy rate, youth total (% of people ages 1...",SE.ADT.1524.LT.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,77.43113,NaN,NaN,NaN
3,Albania,ALB,"Literacy rate, youth total (% of people ages 1...",SE.ADT.1524.LT.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,98.83121,NaN,NaN,98.79119,99.23140,NaN,NaN,NaN,NaN,NaN
4,Andorra,AND,"Literacy rate, youth total (% of people ages 1...",SE.ADT.1524.LT.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00000,100.00000,NaN
5,Arab World,ARB,"Literacy rate, youth total (% of people ages 1...",SE.ADT.1524.LT.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,United Arab Emirates,ARE,"Literacy rate, youth total (% of people ages 1...",SE.ADT.1524.LT.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Argentina,ARG,"Literacy rate, youth total (% of people ages 1...",SE.ADT.1524.LT.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.30000,NaN,NaN
8,Armenia,ARM,"Literacy rate, youth total (% of people ages 1...",SE.ADT.1524.LT.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,99.84740,NaN,NaN,NaN,NaN,NaN,NaN
9,American Samoa,ASM,"Literacy rate, youth total (% of people ages 1...",SE.ADT.1524.LT.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
adult_literacy = pd.read_csv('data/API_SE.ADT.LITR.ZS_DS2_en_csv_v2.csv', header=2)
adult_literacy.head(20) # too many missing values

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,Unnamed: 61
0,Aruba,ABW,"Literacy rate, adult total (% of people ages 1...",SE.ADT.LITR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,96.82264,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,"Literacy rate, adult total (% of people ages 1...",SE.ADT.LITR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,31.74112,NaN,NaN,NaN,NaN,NaN,NaN
2,Angola,AGO,"Literacy rate, adult total (% of people ages 1...",SE.ADT.LITR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,66.03011,NaN,NaN,NaN
3,Albania,ALB,"Literacy rate, adult total (% of people ages 1...",SE.ADT.LITR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,95.93864,NaN,NaN,96.84530,97.24697,NaN,NaN,NaN,NaN,NaN
4,Andorra,AND,"Literacy rate, adult total (% of people ages 1...",SE.ADT.LITR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00000,100.00000,NaN
5,Arab World,ARB,"Literacy rate, adult total (% of people ages 1...",SE.ADT.LITR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,United Arab Emirates,ARE,"Literacy rate, adult total (% of people ages 1...",SE.ADT.LITR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Argentina,ARG,"Literacy rate, adult total (% of people ages 1...",SE.ADT.LITR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.09000,NaN,NaN
8,Armenia,ARM,"Literacy rate, adult total (% of people ages 1...",SE.ADT.LITR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,99.74441,NaN,NaN,NaN,NaN,NaN,NaN
9,American Samoa,ASM,"Literacy rate, adult total (% of people ages 1...",SE.ADT.LITR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
primary_exp = pd.read_csv('data/API_SE.XPD.PRIM.PC.ZS_DS2_en_csv_v2.csv', header=2)
primary_exp.head(20) # too many missing values

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,Unnamed: 61
0,Aruba,ABW,"Government expenditure per student, primary (%...",SE.XPD.PRIM.PC.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,17.966591,17.225630,16.464880,NaN,NaN,NaN
1,Afghanistan,AFG,"Government expenditure per student, primary (%...",SE.XPD.PRIM.PC.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9.445470,9.782870,6.894670,8.997400,9.531700,8.15053,NaN,NaN
2,Angola,AGO,"Government expenditure per student, primary (%...",SE.XPD.PRIM.PC.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Albania,ALB,"Government expenditure per student, primary (%...",SE.XPD.PRIM.PC.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,AND,"Government expenditure per student, primary (%...",SE.XPD.PRIM.PC.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,15.940610,17.124889,17.574381,NaN,14.769330,13.175610,13.96354,NaN,NaN
5,Arab World,ARB,"Government expenditure per student, primary (%...",SE.XPD.PRIM.PC.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,United Arab Emirates,ARE,"Government expenditure per student, primary (%...",SE.XPD.PRIM.PC.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Argentina,ARG,"Government expenditure per student, primary (%...",SE.XPD.PRIM.PC.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,13.675240,14.734910,13.614240,14.155370,14.543320,14.381570,14.371950,NaN,NaN,NaN
8,Armenia,ARM,"Government expenditure per student, primary (%...",SE.XPD.PRIM.PC.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,11.138220,11.339420,NaN,11.36634,NaN,NaN
9,American Samoa,ASM,"Government expenditure per student, primary (%...",SE.XPD.PRIM.PC.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
secondary_exp = pd.read_csv('data/API_SE.XPD.SECO.PC.ZS_DS2_en_csv_v2.csv', header=2)
secondary_exp.head(20) # too many missing values

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,Unnamed: 61
0,Aruba,ABW,"Government expenditure per student, secondary ...",SE.XPD.SECO.PC.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,25.052410,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,"Government expenditure per student, secondary ...",SE.XPD.SECO.PC.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10.479740,10.067680,7.071160,9.115330,9.782210,8.21736,NaN,NaN
2,Angola,AGO,"Government expenditure per student, secondary ...",SE.XPD.SECO.PC.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Albania,ALB,"Government expenditure per student, secondary ...",SE.XPD.SECO.PC.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.867800,NaN,NaN,NaN,NaN
4,Andorra,AND,"Government expenditure per student, secondary ...",SE.XPD.SECO.PC.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,18.052429,13.348440,13.356120,NaN,15.542000,11.408510,14.93023,NaN,NaN
5,Arab World,ARB,"Government expenditure per student, secondary ...",SE.XPD.SECO.PC.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,United Arab Emirates,ARE,"Government expenditure per student, secondary ...",SE.XPD.SECO.PC.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Argentina,ARG,"Government expenditure per student, secondary ...",SE.XPD.SECO.PC.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,18.927389,21.866230,19.157049,20.562740,20.747240,21.675510,21.307051,NaN,NaN,NaN
8,Armenia,ARM,"Government expenditure per student, secondary ...",SE.XPD.SECO.PC.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,15.355620,15.045240,NaN,NaN,NaN
9,American Samoa,ASM,"Government expenditure per student, secondary ...",SE.XPD.SECO.PC.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
poverty_rate = pd.read_csv('data/API_SI.POV.DDAY_DS2_en_csv_v2.csv', header=2)
poverty_rate.head(20) # too many missing values

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,Unnamed: 61
0,Aruba,ABW,Poverty headcount ratio at $1.90 a day (2011 P...,SI.POV.DDAY,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,Poverty headcount ratio at $1.90 a day (2011 P...,SI.POV.DDAY,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Angola,AGO,Poverty headcount ratio at $1.90 a day (2011 P...,SI.POV.DDAY,NaN,NaN,NaN,NaN,NaN,NaN,...,30.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Albania,ALB,Poverty headcount ratio at $1.90 a day (2011 P...,SI.POV.DDAY,NaN,NaN,NaN,NaN,NaN,NaN,...,0.37,NaN,NaN,NaN,1.06,NaN,NaN,NaN,NaN,NaN
4,Andorra,AND,Poverty headcount ratio at $1.90 a day (2011 P...,SI.POV.DDAY,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Arab World,ARB,Poverty headcount ratio at $1.90 a day (2011 P...,SI.POV.DDAY,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,United Arab Emirates,ARE,Poverty headcount ratio at $1.90 a day (2011 P...,SI.POV.DDAY,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Argentina,ARG,Poverty headcount ratio at $1.90 a day (2011 P...,SI.POV.DDAY,NaN,NaN,NaN,NaN,NaN,NaN,...,2.97,3.05,2.05,1.53,1.63,1.75,1.71,NaN,NaN,NaN
8,Armenia,ARM,Poverty headcount ratio at $1.90 a day (2011 P...,SI.POV.DDAY,NaN,NaN,NaN,NaN,NaN,NaN,...,1.49,1.59,2.54,2.70,1.74,2.44,2.31,NaN,NaN,NaN
9,American Samoa,ASM,Poverty headcount ratio at $1.90 a day (2011 P...,SI.POV.DDAY,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
countries=countries.rename(columns={'name_normalized': 'Country Name'})
countries.columns

Index(['id', 'name', 'num_territories', 'region', 'region_id', 'Country Name'], dtype='object')

In [18]:
df = countries.merge(net_migration[['Country Name', '2012']], how='inner')
df = df.rename(columns={'2012': 'net_migration-2012'})

In [19]:
df = df.merge(net_imports[['Country Name', '2012', '2013', '2014', '2015', '2016']], how='inner')

d = {}
for year in ['2012', '2013', '2014', '2015', '2016']:
    d[year] = 'net_imports-'+year
df = df.rename(columns=d)

In [20]:
df = df.merge(net_exports[['Country Name', '2012', '2013', '2014', '2015', '2016']])

d = {}
for year in ['2012', '2013', '2014', '2015', '2016']:
    d[year] = 'net_exports-'+year
df = df.rename(columns=d)

In [22]:
df.head()
df.shape

,id,name,num_territories,region,region_id,Country Name,net_migration-2012,net_imports-2012,net_imports-2013,net_imports-2014,net_imports-2015,net_imports-2016,net_exports-2012,net_exports-2013,net_exports-2014,net_exports-2015,net_exports-2016
0,1,Afghanistan,0,Asia & the Pacific,1,Afghanistan,448007.0,39.142397,49.757775,45.773877,48.801615,49.024977,5.516850,6.307895,6.567700,7.002117,6.896250
1,10,Bangladesh,0,Asia & the Pacific,1,Bangladesh,-2526483.0,27.949334,26.758529,25.524420,24.749324,21.304584,20.161589,19.537874,18.989660,17.336675,16.649717
2,13,Bhutan,0,Asia & the Pacific,1,Bhutan,10000.0,63.038593,62.275571,57.337983,59.972815,52.128654,38.725334,40.459904,36.284637,32.884411,29.406818
3,21,Burma,0,Asia & the Pacific,1,Myanmar,-474278.0,10.886255,18.945661,22.168472,26.536595,26.374178,11.498361,19.635098,20.090340,20.780172,16.411193
4,24,Cambodia,0,Asia & the Pacific,1,Cambodia,-149999.0,62.779410,66.687927,66.708380,66.145643,65.668504,57.955990,61.492924,62.322843,61.718424,61.281525


(131, 17)

(146, 6)

In [23]:
df.to_csv('data/Countries_with_WBdata.csv')